In [4]:
import great_expectations as gx
from great_expectations.core.batch import BatchRequest, RuntimeBatchRequest
from ruamel import yaml

In [5]:
context = gx.get_context()

In [6]:
config = """
name: trino_datasource
class_name: Datasource
execution_engine:
  class_name: SqlAlchemyExecutionEngine
  connection_string: trino://trino@trino:8080/delta/my_schema
data_connectors:
   default_runtime_data_connector_name:
       class_name: RuntimeDataConnector
       batch_identifiers:
           - default_identifier_name
   default_inferred_data_connector_name:
       class_name: InferredAssetSqlDataConnector
       include_schema_name: true
"""

In [7]:
context.test_yaml_config(config)

Attempting to instantiate class from config...
	Instantiating as a Datasource, since class_name is Datasource
	Successfully instantiated Datasource


ExecutionEngine class name: SqlAlchemyExecutionEngine
Data Connectors:
	default_inferred_data_connector_name : InferredAssetSqlDataConnector

	Available data_asset_names (1 of 1):
		my_schema.appl_stock_delta_table (1 of 1): [{}]

	Unmatched data_references (0 of 0):[]

	default_runtime_data_connector_name:RuntimeDataConnector

	Available data_asset_names (0 of 0):
		Note : RuntimeDataConnector will not have data_asset_names until they are passed in through RuntimeBatchRequest

	Unmatched data_references (0 of 0): []



In [8]:
context.add_datasource(**yaml.load(config))

In [9]:
trino = context.get_datasource("trino_datasource")

trino.get_available_data_asset_names()

{'default_runtime_data_connector_name': [],
 'default_inferred_data_connector_name': ['my_schema.appl_stock_delta_table']}

In [16]:
batch_request = BatchRequest(
    datasource_name="trino_datasource",
    data_connector_name="default_inferred_data_connector_name",
    data_asset_name="my_schema.appl_stock_delta_table",  # this is the name of the table you want to retrieve
)
context.create_expectation_suite(
    expectation_suite_name="test_suite", overwrite_existing=True
)
validator = context.get_validator(
    batch_request=batch_request, expectation_suite_name="test_suite"
)

In [22]:
validator.expect_column_to_exist("close")

Calculating Metrics:   0%|          | 0/2 [00:00<?, ?it/s]

{
  "result": {},
  "meta": {},
  "success": true,
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}